# ML Trader

In [ ]:
from utils.packages import *
from utils.variables import *
from utils.i_o import *
from utils.dir_slope import *
from utils.engine import *
from utils.loops import *
from utils.order import *
import winsound

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.graph_objs.layout import YAxis,XAxis,Margin
import webbrowser
data['chrome_path'] = "C:/Program Files (x86)/Google/Chrome/Application/chrome.exe %s"

In [ ]:
weekday_dict = {0:'Sunday' , 1:'Monday', 2:'Tuesday', 3:'Wednesday', 4:'Thursday', 5:'Friday', 6:'Saturday'}

In [ ]:
%%time
data = get_date_list(data)
data = read_data(data)

In [ ]:
%%time
data['df']['tick']      = (data["df"]['Ask'] + data["df"]['Bid'])/2
data['df']['dt']   = [dt.datetime.strptime(x.split(".")[0],"%Y%m%d %H:%M:%S") for x in data["df"]['DateTime']]

data['df'] = data['df'][['dt','tick']]
data['df']['month_val'] = [x.month for x in data['df']['dt']]
data['df']['date_val'] = [x.day for x in data['df']['dt']]
data['df'].head(2)

In [ ]:
%%time
idx = data['df'].groupby(['date_val', 'month_val'])['tick'].transform(max) == data['df']['tick']
data['df_max'] = data['df'][idx]
data['df_max']['week_day'] = [x.weekday() for x in data['df_max']['dt']]
data['df_max'] = data['df_max'].replace({'week_day' : weekday_dict})
del data['df_max']['dt']
data['df_max'] = data['df_max'].drop_duplicates()
data['df_max'].head(2)

In [ ]:
idx = data['df'].groupby(['date_val', 'month_val'])['tick'].transform(min) == data['df']['tick']
data['df_min'] = data['df'][idx]
del data['df_min']['dt']
data['df_min'] = data['df_min'].drop_duplicates()
data['df_min'].head(2)

In [ ]:
data['df_min_max'] = data['df_max'].merge(data['df_min'], on = ['month_val', 'date_val'])
data['df_min_max'] = data['df_min_max'].rename(columns = {'tick_x':'max_tick', 'tick_y':'min_tick'})

In [ ]:
del data['df']
del data['df_min']

In [ ]:
data['df_min_max'] = data['df_min_max'][['month_val', 'date_val', 'week_day','min_tick', 'max_tick']]
data['df_min_max']['day_spread'] = data['df_min_max']['max_tick'] - data['df_min_max']['min_tick']
data['df_min_max']['mid_tick'] = data['df_min_max'][['max_tick', 'min_tick']].mean(axis=1)
data['df_min_max'].head(2)

In [ ]:
data['df_min_max'].groupby('week_day').day_spread.mean()

In [ ]:
x = data['df_min_max'].groupby('week_day').day_spread.mean().index
y = data['df_min_max'].groupby('week_day').day_spread.mean().values
del data['df_min_max']

In [ ]:
chart_name = f"Trade Chart"
layout = go.Layout(title = chart_name,
                       xaxis = dict(title="date"),
                       
                       yaxis = dict(title="PIP"),
                      )
    

fig = go.Figure(layout = layout)



fig.add_trace(go.Bar(x = x,
                     y = y,
                     text = y
                    )
                    )
fig.update_traces(textposition='outside')


fig.show()